In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json
import os
import shutil

In [2]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [3]:
# URL to find information on Coronavirus
url1 = "https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83"

url2 = "https://dshs.texas.gov/coronavirus/"

url3 = "http://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus/Harris-County-COVID-19-Confirmed-Cases"

url4 = "https://www.khou.com/section/news/health/coronavirus"

In [4]:
# Visiting the url 
browser.visit(url1)

In [5]:
time.sleep(7)# Creating the Beautiful soup for URL
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# Revise? 
reported_cases = soup.find_all('text')[9].text
reported_cases

'8,262'

In [7]:
deaths = soup.find_all('text')[11].text
deaths

'154'

In [8]:
df_stats = pd.DataFrame({'Cases':[reported_cases], 'Deaths':[deaths]})
df_stats.to_csv('static/csv/stats.csv', index=False)

In [9]:
# Getting Total Cases and Deaths 
browser.visit(url2)

In [10]:
browser.find_by_text('Accessible version').click()
time.sleep(5)

In [11]:
source = 'C:/Users/zacha/Downloads/'
destination = 'C:/Users/zacha/Documents/COVID-19/'
filename = 'CaseCountData.xlsx'

In [12]:
shutil.move(os.path.join(source, filename), os.path.join(destination, filename))

'C:/Users/zacha/Documents/COVID-19/CaseCountData.xlsx'

In [13]:
# dest = shutil.move(source, destination, )

In [14]:
df = pd.read_excel('CaseCountData.xlsx', header=1).dropna()
df.sort_values(by=['Cases'], ascending=False, inplace=True)
df['County'] = df.apply(lambda x: x['County'].rstrip(), axis=1)
df.to_csv('static/csv/CaseCountData.csv', index=False, header=True)

df

,No.,County,Cases,Fatalities
64,65.0,Harris,1809,22
34,35.0,Dallas,1155,18
140,141.0,Travis,502,6
8,9.0,Bexar,456,12
135,136.0,Tarrant,452,13
...,...,...,...,...
108,109.0,Montague,1,0
111,112.0,Morris,1,0
114,115.0,Newton,1,0
121,122.0,Pecos,1,0


In [15]:
df

,No.,County,Cases,Fatalities
64,65.0,Harris,1809,22
34,35.0,Dallas,1155,18
140,141.0,Travis,502,6
8,9.0,Bexar,456,12
135,136.0,Tarrant,452,13
...,...,...,...,...
108,109.0,Montague,1,0
111,112.0,Morris,1,0
114,115.0,Newton,1,0
121,122.0,Pecos,1,0


In [16]:
browser.visit(url3)

In [17]:
harris_table = browser.find_by_id('covidtable')

In [18]:
html_harris = browser.html
harris_soup = bs(html_harris, 'html.parser')

In [19]:
harris_data = []
table = harris_soup.find('table', attrs={'id':'covidtable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    harris_data.append([ele for ele in cols if ele]) # Get rid of empty values

In [20]:
columns = ['Patient', 'Sex', 'Age_Range', 'Location','Exposure', 'Status']


In [21]:
harris_data = harris_data[1:]

In [22]:
harris_df = pd.DataFrame(harris_data, columns=columns)

In [23]:
harris_df.sort_values(by=['Age_Range'], inplace=True)

In [24]:
harris_df

,Patient,Sex,Age_Range,Location,Exposure,Status
43,44,Male,0 - 9,NW,Possible Exposure to Case,Active
45,46,Female,0 - 9,NW,Possible Exposure to Case,Active
172,173,Female,0 - 9,NW,Possible Exposure to Case,Active
185,186,Female,0 - 9,SE,Possible Exposure to Case,Active
147,148,Male,10 - 19,SE,Possible Exposure to Case,Active
...,...,...,...,...,...,...
154,155,Female,80+,SE,Investigation Pending,Active
246,247,Female,80+,SW,Investigation Pending,Active
15,16,Male,80+,NW,No Known Exposure,Deceased
158,159,Female,80+,NW,History of Travel,Active


In [25]:
harris_df.to_csv('static/csv/harris_stats.csv', index=False, header=True)

In [26]:
harris_df.head()

,Patient,Sex,Age_Range,Location,Exposure,Status
43,44,Male,0 - 9,NW,Possible Exposure to Case,Active
45,46,Female,0 - 9,NW,Possible Exposure to Case,Active
172,173,Female,0 - 9,NW,Possible Exposure to Case,Active
185,186,Female,0 - 9,SE,Possible Exposure to Case,Active
147,148,Male,10 - 19,SE,Possible Exposure to Case,Active


In [27]:
All_Tex_Counties_df = pd.read_csv('static/csv/Texas_Counties.csv')
All_Tex_Counties_df.head()

,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"
3,-96.965687,30.310651,Lee,144,48287,2.213853,0.153990,"(-96.965687, 30.310651)"
4,-95.853568,32.211881,Henderson,108,48213,3.152909,0.235056,"(-95.853568, 32.211881)"


In [28]:
All_Tex_Counties = All_Tex_Counties_df['CNTY_NM'].tolist()

In [29]:
df_counties = df['County'].tolist()

In [30]:
Counties_with_Cases = []

for i in df_counties:
    
    if i.rstrip() in All_Tex_Counties:
        Counties_with_Cases.append(i.rstrip())
        print(f'{i}---found')
        
    elif i == 'DeWitt':
        Counties_with_Cases.append('De Witt')
        print(f'{i}---found')
    else:
        print(f'Houston we have a problem: {i}')

Harris---found
Dallas---found
Travis---found
Bexar---found
Tarrant---found
Denton---found
Fort Bend---found
Collin---found
Galveston---found
Lubbock---found
Brazoria---found
Montgomery---found
El Paso---found
Webb---found
Hidalgo---found
Cameron---found
Brazos---found
Williamson---found
Jefferson---found
Bell---found
Smith---found
Nueces---found
Hays---found
McLennan---found
Victoria---found
Wichita---found
Potter---found
Guadalupe---found
Matagorda---found
Randall---found
Ellis---found
Taylor---found
Ector---found
Hardin---found
Midland---found
Orange---found
Tom Green---found
Johnson---found
Comal---found
Washington---found
Nacogdoches---found
Gregg---found
Bowie---found
Chambers---found
Wharton---found
Bastrop---found
Angelina---found
Grayson---found
Kaufman---found
Shelby---found
Calhoun---found
Liberty---found
Rusk---found
Waller---found
Harrison---found
Rockwall---found
Coryell---found
Castro---found
Walker---found
Andrews---found
Hood---found
Hockley---found
Gray---found
Kendall

In [31]:
All_Tex_Counties_df = All_Tex_Counties_df.set_index('CNTY_NM')

In [32]:
Geo_df = All_Tex_Counties_df.loc[Counties_with_Cases]
Geo_df.head()

,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
CNTY_NM,,,,,,,
Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"


In [33]:
Geo_df = Geo_df.reset_index()
Geo_df = Geo_df.rename(columns={"CNTY_NM": "County"})
Geo_df

,County,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
1,Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
2,Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
4,Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
...,...,...,...,...,...,...,...,...
156,Montague,-97.724670,33.674963,169,48337,2.090266,0.235633,"(-97.72467, 33.674963)"
157,Morris,-94.732051,33.113685,172,48343,1.538698,0.064495,"(-94.732051, 33.113685)"
158,Newton,-93.744630,30.786200,176,48351,3.669102,0.229430,"(-93.74463, 30.7862)"
159,Pecos,-102.723574,30.781000,186,48371,6.107163,1.162561,"(-102.723574, 30.781)"


In [34]:
result = pd.merge(df, Geo_df, on='County')
result = result.rename(columns={'X (Lat)':'long', 'Y (Long)':'lat'})

result.head()

,No.,County,Cases,Fatalities,long,lat,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,65.0,Harris,1809,22,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
1,35.0,Dallas,1155,18,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
2,141.0,Travis,502,6,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,9.0,Bexar,456,12,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
4,136.0,Tarrant,452,13,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"


In [35]:
result = result[['County', 'Cases', 'Fatalities', 'lat', 'long']]
result

,County,Cases,Fatalities,lat,long
0,Harris,1809,22,29.859671,-95.397821
1,Dallas,1155,18,32.766537,-96.777819
2,Travis,502,6,30.334233,-97.781947
3,Bexar,456,12,29.448845,-98.519663
4,Tarrant,452,13,32.771852,-97.291165
...,...,...,...,...,...
155,Montague,1,0,33.674963,-97.724670
156,Morris,1,0,33.113685,-94.732051
157,Newton,1,0,30.786200,-93.744630
158,Pecos,1,0,30.781000,-102.723574


In [36]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lat],row[lon]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['County', 'Cases', 'Fatalities']
javascript_file = df_to_geojson(result, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [37]:
javascript_file

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'County': 'Harris', 'Cases': 1809, 'Fatalities': 22},
   'geometry': {'type': 'Point', 'coordinates': [29.85967144, -95.39782106]}},
  {'type': 'Feature',
   'properties': {'County': 'Dallas', 'Cases': 1155, 'Fatalities': 18},
   'geometry': {'type': 'Point', 'coordinates': [32.76653746, -96.77781862]}},
  {'type': 'Feature',
   'properties': {'County': 'Travis', 'Cases': 502, 'Fatalities': 6},
   'geometry': {'type': 'Point', 'coordinates': [30.33423319, -97.78194654]}},
  {'type': 'Feature',
   'properties': {'County': 'Bexar', 'Cases': 456, 'Fatalities': 12},
   'geometry': {'type': 'Point', 'coordinates': [29.44884523, -98.51966309]}},
  {'type': 'Feature',
   'properties': {'County': 'Tarrant', 'Cases': 452, 'Fatalities': 13},
   'geometry': {'type': 'Point', 'coordinates': [32.77185236, -97.29116473]}},
  {'type': 'Feature',
   'properties': {'County': 'Denton', 'Cases': 337, 'Fatalities': 7},
   'g

In [38]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)

In [39]:
browser.visit(url4)

In [40]:
headlines_harris = browser.html
headline_soup = bs(headlines_harris, 'html.parser')

In [41]:
# lis = headline_soup.find_all('li', class_="headline-list-with-abstract__item")
links = headline_soup.find_all('div', class_="headline-list-with-abstract__headline")

titles = []
hrefs = []

In [42]:
recent = headline_soup.find_all('div', class_="story-snapshot-with-abstract__headline")[0]

In [43]:
recent

<div class="story-snapshot-with-abstract__headline">
<a class="story-snapshot-with-abstract__link" data-tracking="story-snapshot-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/new-data-suggests-staying-home-social-distancing-is-saving-lives/285-880fefea-a7ed-4aad-9d8b-13e8930eeb73">
<div class="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180" style="overflow-wrap: break-word;">
                        New data suggests staying home, social distancing is saving lives
                    </div>
<div class="story-snapshot-with-abstract__mobile-title dot-ellipsis dot-height-135" style="overflow-wrap: break-word;">
                        New data suggests staying home, social distancing is saving lives
                    </div>
</a>
</div>

In [44]:
recent_title = recent.find('div', class_="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180").text.strip()
recent_title

'New data suggests staying home, social distancing is saving lives'

In [45]:
titles.append(recent_title)
hrefs.append('https://www.khou.com' + recent.a['href'])

In [46]:
links

[<div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/schumer-proposes-25000-heroes-pay-for-frontline-workers/285-477230fc-0864-4b9f-937b-038eff6e3412">
                     Schumer proposes $25,000 'heroes' pay for frontline workers
                 </a>
 </div>,
 <div class="headline-list-with-abstract__headline"> <a class="headline-list-with-abstract__link">4 Signs Your Heart Is Quietly Failing You</a> </div>,
 <div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/news/brazos-county-reports-cases-of-coronavirus/285-bb82428a-417e-4287-b7f8-ef6cf60ffa2b">
                     Update: Fourth person dies from COVID-19 in Brazos County, 68 confirmed cases
                 </a>
 </div>,
 <div class="headline-list-with-abst

In [47]:

for article in links:
    
    try:
        
        title = article.find('a', class_="headline-list-with-abstract__link").text
        
        link = article.a['href']
        
        if (title and link):
            
            
            print('-------------')
            print(title)
            print(link)
            titles.append(title.strip())
            hrefs.append('https://www.khou.com' + link)
            
    except:
        print('no')

-------------

                    Schumer proposes $25,000 'heroes' pay for frontline workers
                
/article/news/health/coronavirus/schumer-proposes-25000-heroes-pay-for-frontline-workers/285-477230fc-0864-4b9f-937b-038eff6e3412
no
-------------

                    Update: Fourth person dies from COVID-19 in Brazos County, 68 confirmed cases
                
/article/news/brazos-county-reports-cases-of-coronavirus/285-bb82428a-417e-4287-b7f8-ef6cf60ffa2b
-------------

                    Coronavirus updates: Experts estimate more than 4K lives have been saved due to Harris County stay-home order
                
/article/news/health/coronavirus/coronavirus-updates-houston-live-blog-april-6/285-3fca5511-726f-4157-b3cf-da3a3ee0cfca
-------------

                    Woman who spent days on a ventilator after COVID-19 diagnosis returns home
                
/article/news/health/coronavirus/coronavirus-recovery-st-louis-county-woman-returns-home/285-377860dc-38a1-4384-a2b1-0

In [48]:
latest_headlines_df = pd.DataFrame(list(zip(titles, hrefs)), columns =['Headline', 'link'])

In [49]:
latest_headlines_df

,Headline,link
0,"New data suggests staying home, social distanc...",https://www.khou.com/article/news/health/coron...
1,"Schumer proposes $25,000 'heroes' pay for fron...",https://www.khou.com/article/news/health/coron...
2,Update: Fourth person dies from COVID-19 in Br...,https://www.khou.com/article/news/brazos-count...
3,Coronavirus updates: Experts estimate more tha...,https://www.khou.com/article/news/health/coron...
4,Woman who spent days on a ventilator after COV...,https://www.khou.com/article/news/health/coron...
5,Wisconsin voters head to polls Tuesday despite...,https://www.khou.com/article/news/health/coron...
6,AP sources: MLB considering playing all games ...,https://www.khou.com/article/sports/mlbphoenix...


In [50]:
latest_headlines_df.to_csv('static/csv/headlines.csv', index=False, header=True)

In [51]:
browser.quit()